https://tianchi.aliyun.com/forum/postDetail?spm=5176.12586969.1002.3.657376d8UhQABu&postId=12505

In [35]:
import pandas as pd 
import shap
import sklearn 

# 数据预处理模块

In [23]:
def drop_multiple_col(col_list,df):
    df.drop(col_list,axis=1,inplace=True )

In [33]:
def check_missing(df):
    return df.isnull().sum().sort_values(ascending=False)

# 三张表 资产负债表，现金流量表，利润表

In [2]:
path='/Users/ness001/Downloads/FDDC_financial_data/alike sliver/'

In [3]:
bspath=path+'Balance Sheet.xls'

In [8]:
xlsx=pd.ExcelFile(bspath)

In [9]:
bank=pd.read_excel(xlsx,'Bank')

In [11]:
bank.describe()

,PARTY_ID,TICKER_SYMBOL,FISCAL_PERIOD,MERGED_FLAG,C_RESER_CB,DEPOS_IN_OTH_BFI,PRECI_METALS,LOAN_TO_OTH_BANK_FI,TRADING_FA,DERIV_ASSETS,...,SEE,SEA,T_EQUITY_ATTR_P,MINORITY_INT,OTH_EFFECT_SE,OTH_EFFECT_SA,T_SH_EQUITY,LEE,LEA,T_LIAB_EQUITY
count,1593.000000,1593.000000,1593.000000,1593.0,1.582000e+03,1.581000e+03,7.590000e+02,1.483000e+03,1.480000e+03,1.310000e+03,...,1.280000e+02,34.000000,1.593000e+03,1.314000e+03,4.0,17.000000,1.593000e+03,4.0,17.000000,1.593000e+03
mean,9144.277464,526614.043942,9.894539,1.0,6.410895e+11,1.268640e+11,3.656521e+10,1.063023e+11,6.037300e+10,1.132099e+10,...,6.243744e+09,197.058824,2.915882e+11,4.599441e+09,0.0,294.117647,2.953820e+11,0.0,135.294118,4.454081e+12
std,13426.909467,197891.815615,3.235160,0.0,9.151222e+11,1.639189e+11,5.714003e+10,1.333583e+11,1.134734e+11,2.036140e+10,...,1.813923e+10,948.838299,4.169357e+11,1.096114e+10,0.0,848.874688,4.226342e+11,0.0,694.569270,5.796515e+12
min,2.000000,1.000000,3.000000,1.0,7.162850e+03,1.262089e+08,0.000000e+00,0.000000e+00,1.331156e+06,7.090000e+05,...,-1.929000e+10,-2000.000000,-7.276050e+11,3.000000e+06,0.0,-1000.000000,-7.276050e+11,0.0,-1000.000000,1.600462e+09
25%,1322.000000,600016.000000,9.000000,1.0,6.382775e+10,1.816800e+10,3.544230e+08,8.682559e+09,4.676721e+09,7.638090e+08,...,-1.655000e+08,-100.000000,2.781153e+10,1.617090e+08,0.0,0.000000,2.781153e+10,0.0,0.000000,5.276850e+11
50%,2041.000000,601166.000000,12.000000,1.0,2.563230e+11,6.668491e+10,8.654903e+09,5.638836e+10,1.336000e+10,3.261000e+09,...,1.165305e+08,500.000000,1.146032e+11,7.920000e+08,0.0,1000.000000,1.160780e+11,0.0,0.000000,1.972768e+12
75%,17747.000000,601398.000000,12.000000,1.0,5.533280e+11,1.517150e+11,4.247900e+10,1.461165e+11,5.025700e+10,1.122400e+10,...,1.602952e+09,1000.000000,3.489620e+11,4.018750e+09,0.0,1000.000000,3.520270e+11,0.0,1000.000000,5.541535e+12
max,49847.000000,603323.000000,12.000000,1.0,3.639302e+12,8.045270e+11,2.387140e+11,5.937130e+11,7.681450e+11,1.383600e+11,...,1.021650e+11,1000.000000,2.171510e+12,8.066300e+10,0.0,1000.000000,2.184905e+12,0.0,1000.000000,2.649378e+13


# 对market数据做处理

In [39]:
market=path+'Market Data.xlsx'

In [40]:
xlsx=pd.ExcelFile(market)

In [41]:
market=pd.read_excel(xlsx,'DATA')

In [42]:
market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333138 entries, 0 to 333137
Data columns (total 10 columns):
SECURITY_ID       333138 non-null int64
TICKER_SYMBOL     333138 non-null int64
END_DATE_         333138 non-null datetime64[ns]
CLOSE_PRICE       333138 non-null float64
TURNOVER_VOL      333138 non-null int64
TURNOVER_VALUE    333138 non-null float64
MARKET_VALUE      333138 non-null float64
TYPE_ID           333138 non-null int64
TYPE_NAME_EN      333138 non-null object
TYPE_NAME_CN      333138 non-null object
dtypes: datetime64[ns](1), float64(3), int64(4), object(2)
memory usage: 25.4+ MB


In [44]:
market['TYPE_NAME_CN'].unique()

array(['银行', '金融服务', '房地产', '医药生物', '公用事业', '综合', '休闲服务', '有色金属', '机械设备',
       '建筑装饰', '商业贸易', '建筑材料', '建筑建材', '家用电器', '汽车', '交运设备', '纺织服装',
       '食品饮料', '电子', '计算机', '信息设备', '交通运输', '轻工制造', '通信', '农林牧渔', '化工',
       '传媒', '钢铁', '信息服务', '采掘', '电气设备', '非银金融', '国防军工'], dtype=object)

In [47]:
market['TYPE_ID'].unique()

array([10303210101, 10303330101, 10303200101, 10303170301, 10303180401,
       10303280101, 10303240401, 10303050304, 10303050505, 10303240201,
       10303090501, 10303050301, 10303070301, 10303230401, 10303060201,
       10303290101, 10303130201, 10303110401, 10303310305, 10303310301,
       10303070201, 10303150104, 10303140204, 10303120501, 10303120203,
       10303250101, 10303320201, 10303190101, 10303060303, 10303290104,
       10303290201, 10303290205, 10303110301, 10303310401, 10303160303,
       10303160302, 10303300301, 10303180101, 10303170401, 10303110201,
       10303310201, 10303160305, 10303270201, 10303320102, 10303320101,
       10303250202, 10303010401, 10303030101, 10303180103, 10303260201,
       10303090401, 10303300501, 10303040201, 10303120301, 10303150106,
       10303150102, 10303010601, 10303120401, 10303030701, 10303070504,
       10303290203, 10303230301, 10303230503, 10303230501, 10303230302,
       10303030301, 10303050103, 10303050503, 10303050506, 10303

In [48]:
drop_list=['TYPE_ID','TYPE_NAME_CN']

In [49]:
drop_multiple_col(drop_list,market)

In [53]:
check_missing(market)

TYPE_NAME_EN      0
MARKET_VALUE      0
TURNOVER_VALUE    0
TURNOVER_VOL      0
CLOSE_PRICE       0
END_DATE_         0
TICKER_SYMBOL     0
SECURITY_ID       0
dtype: int64

In [51]:
from sklearn.preprocessing import LabelEncoder

In [52]:
encoder=LabelEncoder()

In [55]:
market_encoded=encoder.fit_transform(market['TYPE_NAME_EN'])